In [1]:
"""
Created on August 4th 2022

@author: JJ
"""

'\nCreated on August 4th 2022\n\n@author: JJ\n'

In [2]:
import os, re, sys
import numpy as np
import pandas as pd
from math import log
import random
import scipy.io
import tensorflow as tf
import csv

2022-12-06 14:17:16.493483: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.constraints import *
from tensorflow.keras.regularizers import *
#from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import (
    activations, 
    initializers, 
    regularizers, 
    constraints
)

In [4]:
num_threads = 8
# Maximum number of threads to use for OpenMP parallel regions.
os.environ["OMP_NUM_THREADS"] = "8"
# Without setting below 2 environment variables, it didn't work for me. Thanks to @cjw85 
os.environ["TF_NUM_INTRAOP_THREADS"] = "4"
os.environ["TF_NUM_INTEROP_THREADS"] = "4"

tf.config.threading.set_inter_op_parallelism_threads(
    num_threads
)
tf.config.threading.set_intra_op_parallelism_threads(
    num_threads
)
tf.config.set_soft_device_placement(True)

In [5]:
# in a well-defined initial state.
random.seed
np.random.seed(1337)
tf.random.set_seed(1337)

In [6]:
import matplotlib
import matplotlib.pyplot as plt

In [7]:
from tensorflow.keras.callbacks import (
    ModelCheckpoint, 
    EarlyStopping
)
from sklearn.metrics import (
    roc_curve,
    auc,
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
)

In [8]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import resample, shuffle
from sklearn.feature_selection import (
    SelectKBest,
    chi2
)
import keras_tuner as kt
from collections import defaultdict

In [9]:
import pickle as pkl

In [10]:
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

In [11]:
pretrained_model_generator, input_encoder = load_pretrained_model('/home/jjia1/viralepitope/viralepitope', 'epoch_92400_sample_23500000.pkl')

In [12]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [13]:
allpep = pd.read_csv('viralepitope_sequenceonly.csv')
allmhc = pd.read_csv('mhcepitope_sequenceonly.csv')

In [14]:
all_pep = allpep.drop(allpep.columns[[0]], axis = 1)
all_mhc = allmhc.drop(allmhc.columns[[0]], axis = 1)

In [15]:
train_pep, test_pep = train_test_split(all_pep, stratify = all_pep['label'], test_size = 0.1, random_state = 0)
train_mhc, test_mhc = train_test_split(all_mhc, stratify = all_mhc['label'], test_size = 0.1, random_state = 0)

In [16]:
train_pep, val_pep = train_test_split(train_pep, stratify = train_pep['label'], test_size = 0.1, random_state = 0)
train_mhc, val_mhc = train_test_split(train_mhc, stratify = train_mhc['label'], test_size = 0.1, random_state = 0)

In [17]:
train_pep.columns

Index(['sequence', 'label'], dtype='object')

In [18]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

training_callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
    tf.keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
]

In [19]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1337)    
allprobas_=np.array([]) 
all_labels=np.array([])

In [20]:
#for i, (train, test) in enumerate(kfold.split(train_pep['sequence'], train_pep['label'])):
 #   training_pep = train_pep[train]
 #   training_mhc = train_mhc[train]
    
#    validation_pep = train_pep[test]
#    validation_mhc = train_mhc[test]
    

finetune(model_generator, input_encoder, OUTPUT_SPEC, train_pep['sequence'], train_pep['label'], val_pep['sequence'], val_pep['label'], \
        seq_len = 8, batch_size = 128, max_epochs_per_stage = 30, lr = 1e-03, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 16, final_lr = 1e-05, callbacks = training_callbacks)

[2022_12_06-14:18:14] Training set: Filtered out 0 of 765711 (0.0%) records of lengths exceeding 30.
[2022_12_06-14:18:20] Validation set: Filtered out 0 of 85080 (0.0%) records of lengths exceeding 30.
[2022_12_06-14:18:20] Training with frozen pretrained layers...


2022-12-06 14:18:20.546130: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-06 14:18:20.546571: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-06 14:18:20.546591: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-06 14:18:20.546617: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sbcphaplp001.uthouston.edu): /proc/driver/nvidia/version does not exist
2022-12-06 14:18:20.546830: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other op

Epoch 1/30
5983/5983 [==============================] - 869s 144ms/step - loss: 0.2843 - val_loss: 0.2901
Epoch 2/30
5983/5983 [==============================] - 1363s 228ms/step - loss: 0.2847 - val_loss: 0.2825
Epoch 3/30
5983/5983 [==============================] - 987s 165ms/step - loss: 0.2846 - val_loss: 0.2447
Epoch 4/30
5983/5983 [==============================] - 879s 147ms/step - loss: 0.2831 - val_loss: 0.2433
Epoch 5/30
5983/5983 [==============================] - 864s 144ms/step - loss: 0.2862 - val_loss: 0.2518

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 6/30
5983/5983 [==============================] - 848s 142ms/step - loss: 0.2580 - val_loss: 0.2351
Epoch 7/30
5983/5983 [==============================] - 844s 141ms/step - loss: 0.2510 - val_loss: 0.2343
Epoch 8/30
5983/5983 [==============================] - 852s 142ms/step - loss: 0.2513 - val_loss: 0.2324
Epoch 9/30
5983/5983 [==============================] - 852s 142ms/ste

2022-12-06 17:49:01.624850: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 6847753473 exceeds 10% of free system memory.


Epoch 1/30
5983/5983 [==============================] - 2656s 443ms/step - loss: 0.2032 - val_loss: 0.1749
Epoch 2/30
5983/5983 [==============================] - 2636s 441ms/step - loss: 0.1625 - val_loss: 0.1441
Epoch 3/30
5983/5983 [==============================] - 2616s 437ms/step - loss: 0.1262 - val_loss: 0.1263
Epoch 4/30
5983/5983 [==============================] - 2606s 436ms/step - loss: 0.1027 - val_loss: 0.1159
Epoch 5/30
5983/5983 [==============================] - 2608s 436ms/step - loss: 0.0859 - val_loss: 0.1051
Epoch 6/30
5983/5983 [==============================] - 2593s 433ms/step - loss: 0.0740 - val_loss: 0.1011
Epoch 7/30
5983/5983 [==============================] - 2636s 441ms/step - loss: 0.0644 - val_loss: 0.1173

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 8/30
5983/5983 [==============================] - 2555s 427ms/step - loss: 0.0336 - val_loss: 0.0909
Epoch 9/30
5983/5983 [==============================] - 2340s 3

2022-12-07 00:54:02.458241: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3136352256 exceeds 10% of free system memory.
2022-12-07 00:54:04.612983: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 6847753473 exceeds 10% of free system memory.


191428/191428 [==============================] - 53511s 280ms/step - loss: 0.0208 - val_loss: 0.1123


In [28]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_pep['sequence'], test_pep['label'], \
        start_seq_len = 16, start_batch_size = 64)

In [29]:
print('Test-set performance:')
display(results)

Test-set performance:


,# records,AUC
Model seq len,,
16,94430,0.978072
32,103,0.814433
All,94533,0.977997


In [30]:
print('Confusion matrix:')
display(confusion_matrix)

Confusion matrix:


,0,1
0,84602,1338
1,1263,7330


In [33]:
model = model_generator.create_model(16)

In [35]:
model.save_weights('model/protein_bert.h5')

In [ ]:
pep_sgtembedding.drop(columns = 'id', inplace = True)
mhc_sgtembedding.drop(columns = 'id', inplace = True)

In [ ]:
#pep_sgtembedding, mhc_sgtembedding, encoded_target
train_pep, test_pep, train_mhc, test_mhc, train_target, test_target = train_test_split(pep_sgtembedding, mhc_sgtembedding, target_df, test_size = 0.2, stratify = target_df)

In [ ]:
print(train_pep.shape)
print(test_pep.shape)

In [ ]:
train_pep

In [ ]:
train_target

In [ ]:
def build_model(training_pep, training_mhc):
    embed_dim = 32
    ff_dim = 32
    num_heads = 2

    input_pep = Input(shape=(training_pep.shape[1:]), name = 'peptide')
    input_mhc = Input(shape=(training_pep.shape[1:]), name = 'mhc')

    transformer_pep = TransformerBlock(embed_dim, num_heads, ff_dim)
    transformer_mhc = TransformerBlock(embed_dim, num_heads, ff_dim)

    x_pep = transformer_pep(input_pep)
    x_mhc = transformer_mhc(input_mhc)

    x_pep = GlobalAveragePooling1D()(x_pep)
    x_mhc = GlobalAveragePooling1D()(x_mhc)

    x_pep = Dropout(0.1)(x_pep)
    x_mhc = Dropout(0.1)(x_mhc)

    x_pep = Dense(20, activation = 'relu')(x_pep)
    x_mhc = Dense(20, activation = 'relu')(x_mhc)

    x_pep = Dropout(0.1)(x_pep)
    x_mhc = Dropout(0.1)(x_mhc)

    output_pep = Dense(1)(x_pep)
    output_mhc = Dense(1)(x_mhc)

    combined_output = concatenate([output_pep, output_mhc])
    combinedDense_output = Dense(1)(combined_output)
    output = Activation('sigmoid')(combinedDense_output)

    model = Model(inputs = [input_pep, input_mhc], outputs = output)
    opt = Adam(learning_rate = 1e-3)
    model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy', 'AUC'])
    model.summary()
    return model

In [ ]:
encoder = LabelEncoder()
encoder.fit(train_target)
encoded_y = encoder.transform(train_target)

In [ ]:
x_pep = pd.DataFrame(train_pep)
x_mhc = pd.DataFrame(train_mhc)

In [ ]:
z = x_pep.iloc[1:10]
z_n = z.to_numpy()

In [ ]:
folder = 'transformer/model/'
if not os.path.isdir(folder):
        os.makedirs(folder)

for i, (train_index, test_index) in enumerate(kfold.split(x_pep, encoded_y)):
    training_pep = x_pep.iloc[train_index]
    training_mhc = x_mhc.iloc[train_index]
    training_target = encoded_y[train_index]
    
    validation_pep = x_pep.iloc[test_index]
    validation_mhc = x_mhc.iloc[test_index]
    validation_target = encoded_y[test_index]
    
    training_pep = training_pep.to_numpy()
    training_mhc = training_mhc.to_numpy()
    validation_pep = validation_pep.to_numpy()
    validation_mhc = validation_mhc.to_numpy()
        
    callbacks_list = [
                ModelCheckpoint(filepath=(folder+'CV/model_%s.h5' %i), monitor='val_accuracy', mode='max', verbose=1, save_best_only=True),
                EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 5)
        ]
    model = build_model(training_pep, training_mhc)
    model.summary()

    model.fit([training_pep,training_mhc], 
            training_target,
            batch_size=512,
            epochs = 100,
            shuffle=True,
            callbacks=callbacks_list,
            validation_data=([validation_pep,validation_mhc], validation_target),
            verbose=1)  
    del model

    saved_model = build_model(training_pep, training_mhc)
    saved_model.load_weights('model/CV/model_' +str(i)+'.h5')
    probas_ = saved_model.predict([np.array(validation_pep),np.array(validation_mhc)])
    allprobas_ = np.append(allprobas_, probas_)           
    all_labels = np.append(all_labels, validation_target)
    del saved_model


In [ ]:
font1 = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 16}
figsize=6.2, 6.2

for i, (train, test) in enumerate(kfold.split(train_pep, train_target)):
    training_pep = train_pep[train]
    training_mhc = train_mhc[train]
    training_target = train_target[train]
    
    validation_pep = train_pep[test]
    validation_mhc = train_mhc[test]
    validation_target = train_target[test]

    if i == 0:
        saved_model = build_model(training_pep, training_mhc)
        saved_model.load_weights('model/CV/model_' +str(i)+'.h5')
        probas_ = saved_model.predict([np.array(validation_pep),np.array(validation_mhc)])
        allprobas_ = np.append(allprobas_, probas_)           
        all_labels = np.append(all_labels, validation_target)
        del saved_model

        fpr, tpr, thresholds = roc_curve(all_labels, allprobas_)
        roc_auc = auc(fpr, tpr)
        precision, recall, _ = precision_recall_curve(all_labels, allprobas_)
        aupr = average_precision_score(all_labels, allprobas_)
        print(roc_auc)
        print(aupr)

        figure1, ax1 = plt.subplots(figsize=figsize)
        ax1.tick_params(labelsize=18)
        labels = ax1.get_xticklabels() + ax1.get_yticklabels()
        [label.set_fontname('Times New Roman') for label in labels]  
        ax1.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Luck', alpha=.8)

        ax1.plot(fpr, tpr, color='b',
                label=r'First CV ROC (AUC = %0.4f)' % (roc_auc),
                    lw=2, alpha=.8)
        ax1.set_xlim([-0.05, 1.05])
        ax1.set_ylim([-0.05, 1.05])
        ax1.set_xlabel('False Positive Rate', font1)
        ax1.set_ylabel('True Positive Rate', font1)
        title1 = 'Cross Validated ROC Curve'
        ax1.set_title(title1, font1)
        ax1.legend(loc="lower right")
        
        ########PR_figure
        figure2, ax2 = plt.subplots(figsize=figsize)
        ax2.tick_params(labelsize=18)
        labels = ax2.get_xticklabels() + ax2.get_yticklabels()
        [label.set_fontname('Times New Roman') for label in labels] 

        ax2.plot(recall, precision, color='b',
                label=r'First CV Precision-Recall (AUC = %0.4f)' % aupr,
                lw=2, alpha=.8)

        ax2.set_xlim([-0.05, 1.05])
        ax2.set_ylim([-0.05, 1.05])
        ax2.set_xlabel('Recall', font1)
        ax2.set_ylabel('Precision', font1)
        title2 = 'Cross Validated PR Curve'
        ax2.set_title(title2, font1)
        ax2.legend(loc="lower left")

    else:
        saved_model = build_model(training_pep, training_mhc)
        saved_model.load_weights('model/CV/model_' +str(i)+'.h5')
        probas_ = saved_model.predict([np.array(validation_pep),np.array(validation_mhc)])
        allprobas_ = np.append(allprobas_, probas_)           
        all_labels = np.append(all_labels, validation_target)
        del saved_model
        
        fpr, tpr, thresholds = roc_curve(validation_target, probas_)
        roc_auc = auc(fpr, tpr)
        precision, recall, _ = precision_recall_curve(validation_target, probas_)
        aupr = average_precision_score(validation_target, probas_)
        print(roc_auc)
        print(aupr)

        ax1.plot(fpr, tpr,
        label=str(i)+'th CV ROC (AUC = %0.4f)' % (roc_auc),
            lw=2, alpha=.8)
        ax1.legend(loc="lower right")


        ax2.plot(recall, precision, 
        label=str(i)+'th CV Precision-Recall (AUC = %0.4f)' % (aupr),
        lw=2, alpha=.8)
        ax2.legend(loc="lower left")

In [ ]:
figure1.savefig('figures/5_fold_roc.jpg', dpi=300, bbox_inches = 'tight')
figure2.savefig('figures/5_fold_prc.jpg', dpi=300, bbox_inches = 'tight')

In [ ]:
font1 = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 16}
figsize=6.2, 6.2

########ROC_figure
figure1, ax1 = plt.subplots(figsize=figsize)
ax1.tick_params(labelsize=18)
labels = ax1.get_xticklabels() + ax1.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]  

fpr, tpr, thresholds = roc_curve(all_labels, allprobas_)
roc_auc = auc(fpr, tpr)
print(roc_auc)

ax1.plot(fpr, tpr, color='b',
    label=r'Mean ROC (AUC = %0.4f)' % (roc_auc),
    lw=2, alpha=.8)
ax1.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Luck', alpha=.8)
ax1.set_xlim([-0.05, 1.05])
ax1.set_ylim([-0.05, 1.05])
ax1.set_xlabel('False Positive Rate', font1)
ax1.set_ylabel('True Positive Rate', font1)
title1 = 'Cross Validated ROC Curve'
ax1.set_title(title1, font1)
ax1.legend(loc="lower right")
figure1.savefig('figures/mean_5_fold_roc.jpg', dpi=300, bbox_inches = 'tight')

########PR_figure
figure2, ax2 = plt.subplots(figsize=figsize)
ax2.tick_params(labelsize=18)
labels = ax2.get_xticklabels() + ax2.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels] 

precision, recall, _ = precision_recall_curve(all_labels, allprobas_)
ax2.plot(recall, precision, color='b',
        label=r'Mean Precision-Recall (AUC = %0.4f)' % (average_precision_score(all_labels, allprobas_)),
        lw=2, alpha=.8)

ax2.set_xlim([-0.05, 1.05])
ax2.set_ylim([-0.05, 1.05])
ax2.set_xlabel('Recall', font1)
ax2.set_ylabel('Precision', font1)
title2 = 'Cross Validated PR Curve'
ax2.set_title(title2, font1)
ax2.legend(loc="lower left")
figure2.savefig('figures/mean_5_fold_pr.jpg', dpi=300, bbox_inches = 'tight')


In [ ]:
allprobas2_=np.array([]) 
all_labels2=np.array([])

In [ ]:
font1 = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 16}
figsize=6.2, 6.2

for i in range(5):
    if i == 0:
        saved_model = build_model(training_pep, training_mhc)
        saved_model.load_weights('model/CV/model_' +str(i)+'.h5')
        probas_ = saved_model.predict([np.array(test_pep),np.array(test_mhc)])
        allprobas2_ = np.append(allprobas2_, probas_)           
        all_labels2 = np.append(all_labels2, test_target)
        del saved_model

        fpr, tpr, thresholds = roc_curve(all_labels2, allprobas2_)
        roc_auc = auc(fpr, tpr)
        precision, recall, _ = precision_recall_curve(all_labels2, allprobas2_)
        aupr = average_precision_score(all_labels2, allprobas2_)
        print(roc_auc)
        print(aupr)

        figure1, ax1 = plt.subplots(figsize=figsize)
        ax1.tick_params(labelsize=18)
        labels = ax1.get_xticklabels() + ax1.get_yticklabels()
        [label.set_fontname('Times New Roman') for label in labels]  
        ax1.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Luck', alpha=.8)

        ax1.plot(fpr, tpr, color='b',
                label=r'First CV ROC (AUC = %0.4f)' % (roc_auc),
                    lw=2, alpha=.8)
        ax1.set_xlim([-0.05, 1.05])
        ax1.set_ylim([-0.05, 1.05])
        ax1.set_xlabel('False Positive Rate', font1)
        ax1.set_ylabel('True Positive Rate', font1)
        title1 = 'Cross Validated ROC Curve'
        ax1.set_title(title1, font1)
        ax1.legend(loc="lower right")
        
        ########PR_figure
        figure2, ax2 = plt.subplots(figsize=figsize)
        ax2.tick_params(labelsize=18)
        labels = ax2.get_xticklabels() + ax2.get_yticklabels()
        [label.set_fontname('Times New Roman') for label in labels] 

        ax2.plot(recall, precision, color='b',
                label=r'First CV Precision-Recall (AUC = %0.4f)' % aupr,
                lw=2, alpha=.8)

        ax2.set_xlim([-0.05, 1.05])
        ax2.set_ylim([-0.05, 1.05])
        ax2.set_xlabel('Recall', font1)
        ax2.set_ylabel('Precision', font1)
        title2 = 'Cross Validated PR Curve'
        ax2.set_title(title2, font1)
        ax2.legend(loc="lower left")

    else:
        saved_model = build_model(training_pep, training_mhc)
        saved_model.load_weights('model/CV/model_' +str(i)+'.h5')
        probas_ = saved_model.predict([np.array(test_pep),np.array(test_mhc)])
        allprobas2_ = np.append(allprobas2_, probas_)           
        all_labels2 = np.append(all_labels2, test_target)
        del saved_model
        
        fpr, tpr, thresholds = roc_curve(test_target, probas_)
        roc_auc = auc(fpr, tpr)
        precision, recall, _ = precision_recall_curve(test_target, probas_)
        aupr = average_precision_score(test_target, probas_)
        print(roc_auc)
        print(aupr)

        ax1.plot(fpr, tpr,
        label=str(i)+'th CV ROC (AUC = %0.4f)' % (roc_auc),
            lw=2, alpha=.8)
        ax1.legend(loc="lower right")


        ax2.plot(recall, precision, 
        label=str(i)+'th CV Precision-Recall (AUC = %0.4f)' % (aupr),
        lw=2, alpha=.8)
        ax2.legend(loc="lower left")

figure1.savefig('figures/test_5_fold_roc.jpg', dpi=300, bbox_inches = 'tight')
figure2.savefig('figures/test_5_fold_prc.jpg', dpi=300, bbox_inches = 'tight')

In [ ]:
font1 = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 16}
figsize=6.2, 6.2

########ROC_figure
figure1, ax1 = plt.subplots(figsize=figsize)
ax1.tick_params(labelsize=18)
labels = ax1.get_xticklabels() + ax1.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]  

fpr, tpr, thresholds = roc_curve(all_labels2, allprobas2_)
roc_auc = auc(fpr, tpr)
print(roc_auc)

ax1.plot(fpr, tpr, color='b',
    label=r'Mean ROC (AUC = %0.4f)' % (roc_auc),
    lw=2, alpha=.8)
ax1.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Luck', alpha=.8)
ax1.set_xlim([-0.05, 1.05])
ax1.set_ylim([-0.05, 1.05])
ax1.set_xlabel('False Positive Rate', font1)
ax1.set_ylabel('True Positive Rate', font1)
title1 = 'Cross Validated ROC Curve'
ax1.set_title(title1, font1)
ax1.legend(loc="lower right")
figure1.savefig('figures/test_mean_5_fold_roc.jpg', dpi=300, bbox_inches = 'tight')

########PR_figure
figure2, ax2 = plt.subplots(figsize=figsize)
ax2.tick_params(labelsize=18)
labels = ax2.get_xticklabels() + ax2.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels] 

precision, recall, _ = precision_recall_curve(all_labels2, allprobas2_)
ax2.plot(recall, precision, color='b',
        label=r'Mean Precision-Recall (AUC = %0.4f)' % (average_precision_score(all_labels2, allprobas2_)),
        lw=2, alpha=.8)

ax2.set_xlim([-0.05, 1.05])
ax2.set_ylim([-0.05, 1.05])
ax2.set_xlabel('Recall', font1)
ax2.set_ylabel('Precision', font1)
title2 = 'Cross Validated PR Curve'
ax2.set_title(title2, font1)
ax2.legend(loc="lower left")
figure2.savefig('figures/test_mean_5_fold_pr.jpg', dpi=300, bbox_inches = 'tight')


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
figure1, ax1 = plt.subplots(figsize=figsize)
figure2, ax2 = plt.subplots(figsize=figsize)
for i in range(5):
    saved_model = build_model(training_pep, training_mhc)
    saved_model = load_model('model/CV/model_'+str(i)+'.h5')

    ax1.plot(saved_model.history["loss"])
    ax1.plot(saved_model.history["val_loss"])
    ax1.legend(['train', 'test'], loc='best')
 
    ax2.plot(saved_model.history["accuracy"])
    ax2.plot(saved_model.history["val_accuracy"])
    ax2.legend(['train', 'test'], loc='best')

    del saved_model
